<a href="https://colab.research.google.com/github/Houdini24/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT YEAR(CAST(date AS DATE)) AS year_sold,
format_number(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms=4
GROUP BY YEAR(CAST(date AS DATE))
ORDER BY YEAR(CAST(date AS DATE))
""").show()


+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2019|   300,263.70|
|     2020|   298,353.78|
|     2021|   301,819.44|
|     2022|   296,363.88|
+---------+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT date_built, format_number(AVG(price), 2) AS average_price, bedrooms, bathrooms
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built, bedrooms, bathrooms
ORDER BY date_built""").show()


+----------+-------------+--------+---------+
|date_built|average_price|bedrooms|bathrooms|
+----------+-------------+--------+---------+
|      2010|   292,859.62|       3|        3|
|      2011|   291,117.47|       3|        3|
|      2012|   293,683.19|       3|        3|
|      2013|   295,962.27|       3|        3|
|      2014|   290,852.27|       3|        3|
|      2015|   288,770.30|       3|        3|
|      2016|   290,555.07|       3|        3|
|      2017|   292,676.79|       3|        3|
+----------+-------------+--------+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
#from pyspark.sql.functions import round
spark.sql("""
SELECT date_built, format_number(AVG(price), 2) AS average_price, bedrooms, bathrooms, floors
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built, bedrooms, bathrooms, floors
ORDER BY date_built""").show()





+----------+-------------+--------+---------+------+
|date_built|average_price|bedrooms|bathrooms|floors|
+----------+-------------+--------+---------+------+
|      2010|   285,010.22|       3|        3|     2|
|      2011|   276,553.81|       3|        3|     2|
|      2012|   307,539.97|       3|        3|     2|
|      2013|   303,676.79|       3|        3|     2|
|      2014|   298,264.72|       3|        3|     2|
|      2015|   297,609.97|       3|        3|     2|
|      2016|   293,965.10|       3|        3|     2|
|      2017|   280,317.58|       3|        3|     2|
+----------+-------------+--------+---------+------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql("""
SELECT format_number(AVG(price), 2) AS average_price, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
| 1,026,669.50| 100|
| 1,061,201.42|  99|
| 1,053,739.33|  98|
| 1,129,040.15|  97|
| 1,017,815.92|  96|
| 1,054,325.60|  95|
| 1,033,536.20|  94|
| 1,026,006.06|  93|
|   970,402.55|  92|
| 1,137,372.73|  91|
| 1,062,654.16|  90|
| 1,107,839.15|  89|
| 1,031,719.35|  88|
| 1,072,285.20|  87|
| 1,070,444.25|  86|
| 1,056,336.74|  85|
| 1,117,233.13|  84|
| 1,033,965.93|  83|
| 1,063,498.00|  82|
| 1,053,472.79|  81|
+-------------+----+
only showing top 20 rows

--- 1.460801124572754 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

spark.sql("""
SELECT format_number(AVG(price), 2) AS average_price, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|average_price|view|
+-------------+----+
| 1,026,669.50| 100|
| 1,061,201.42|  99|
| 1,053,739.33|  98|
| 1,129,040.15|  97|
| 1,017,815.92|  96|
| 1,054,325.60|  95|
| 1,033,536.20|  94|
| 1,026,006.06|  93|
|   970,402.55|  92|
| 1,137,372.73|  91|
| 1,062,654.16|  90|
| 1,107,839.15|  89|
| 1,031,719.35|  88|
| 1,072,285.20|  87|
| 1,070,444.25|  86|
| 1,056,336.74|  85|
| 1,117,233.13|  84|
| 1,033,965.93|  83|
| 1,063,498.00|  82|
| 1,053,472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.6069457530975342 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [ ]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT format_number(AVG(price), 2) AS average_price, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
| 1,026,669.50| 100|
| 1,061,201.42|  99|
| 1,053,739.33|  98|
| 1,129,040.15|  97|
| 1,017,815.92|  96|
| 1,054,325.60|  95|
| 1,033,536.20|  94|
| 1,026,006.06|  93|
|   970,402.55|  92|
| 1,137,372.73|  91|
| 1,062,654.16|  90|
| 1,107,839.15|  89|
| 1,031,719.35|  88|
| 1,072,285.20|  87|
| 1,070,444.25|  86|
| 1,056,336.74|  85|
| 1,117,233.13|  84|
| 1,033,965.93|  83|
| 1,063,498.00|  82|
| 1,053,472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.573807954788208 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("Whoops, it looks like home_sales is still cached.")
else:
  print("Yipee, home_sales is no longer cached!")


Yipee, home_sales is no longer cached!
